# NYC Trees

## Import all packages

In [1]:
import pandas as pd
import plotly.express as px
from jupyter_dash import JupyterDash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output
import matplotlib.pyplot as plt
import numpy as np

## Create histogram of all NYC by Borough

In [2]:
trees_df = pd.read_csv('2015-street-tree-census-tree-data.csv')

In [3]:
trees_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 683788 entries, 0 to 683787
Data columns (total 45 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   tree_id           683788 non-null  int64  
 1   block_id          683788 non-null  int64  
 2   created_at        683788 non-null  object 
 3   tree_dbh          683788 non-null  int64  
 4   stump_diam        683788 non-null  int64  
 5   curb_loc          683788 non-null  object 
 6   status            683788 non-null  object 
 7   health            652172 non-null  object 
 8   spc_latin         652169 non-null  object 
 9   spc_common        652169 non-null  object 
 10  steward           652173 non-null  object 
 11  guards            652172 non-null  object 
 12  sidewalk          652172 non-null  object 
 13  user_type         683788 non-null  object 
 14  problems          652124 non-null  object 
 15  root_stone        683788 non-null  object 
 16  root_grate        68

In [4]:
#clean null values in species latin and common names in preparation for historgram 
trees_df['spc_latin'].fillna("Unknown", inplace = True)
trees_df['spc_common'].fillna("Unknown", inplace = True)

In [5]:
# increase max columns and rows for viewing large data sets
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

In [6]:
#format species common name with capitalized first letter of each word
trees_df['spc_common'] = trees_df['spc_common'].str.title()

In [7]:
#create Genus column
trees_df['Genus'] = trees_df['spc_latin'].str.split(' ').str[0]

In [8]:
trees_df.tail()

,tree_id,block_id,created_at,tree_dbh,stump_diam,curb_loc,status,health,spc_latin,spc_common,steward,guards,sidewalk,user_type,problems,root_stone,root_grate,root_other,trunk_wire,trnk_light,trnk_other,brch_light,brch_shoe,brch_other,address,postcode,zip_city,community board,borocode,borough,cncldist,st_assem,st_senate,nta,nta_name,boro_ct,state,latitude,longitude,x_sp,y_sp,council district,census tract,bin,bbl,Genus
683783,155433,217978,2015-08-18T00:00:00.000,25,0,OnCurb,Alive,Good,Quercus palustris,Pin Oak,None,None,Damage,Volunteer,None,No,No,No,No,No,No,No,No,No,32 MARCY AVENUE,11211,Brooklyn,301,3,Brooklyn,34,53,18,BK73,North Side-South Side,3051900,New York,40.713211,-73.954944,9.967407e+05,199121.6363,34.0,519.0,3062513.0,3.023690e+09,Quercus
683784,183795,348185,2015-08-29T00:00:00.000,7,0,OnCurb,Alive,Good,Cladrastis kentukea,Kentucky Yellowwood,1or2,None,NoDamage,Volunteer,None,No,No,No,No,No,No,No,No,No,67-035 SELFRIDGE STREET,11375,Forest Hills,406,4,Queens,29,28,15,QN17,Forest Hills,4070700,New York,40.715194,-73.856650,1.023989e+06,199873.6475,29.0,707.0,4075448.0,4.031810e+09,Cladrastis
683785,166161,401670,2015-08-22T00:00:00.000,12,0,OnCurb,Alive,Good,Acer rubrum,Red Maple,None,None,Damage,Volunteer,None,No,No,No,No,No,No,No,No,No,130 BIDWELL AVENUE,10314,Staten Island,501,5,Staten Island,50,63,24,SI07,Westerleigh,5020100,New York,40.620762,-74.136517,9.463514e+05,165466.0763,50.0,201.0,5011657.0,5.004080e+09,Acer
683786,184028,504204,2015-08-29T00:00:00.000,9,0,OnCurb,Alive,Good,Acer rubrum,Red Maple,None,None,NoDamage,TreesCount Staff,None,No,No,No,No,No,No,No,No,No,1985 ANTHONY AVENUE,10457,Bronx,205,2,Bronx,15,86,33,BX41,Mount Hope,2023502,New York,40.850828,-73.903115,1.011054e+06,249271.9507,15.0,23502.0,2007757.0,2.028120e+09,Acer
683787,200607,306527,2015-09-03T00:00:00.000,23,0,OnCurb,Alive,Fair,Acer rubrum,Red Maple,None,None,NoDamage,TreesCount Staff,None,No,No,No,No,No,No,No,No,No,69-069 183 STREET,11365,Fresh Meadows,408,4,Queens,24,25,11,QN41,Fresh Meadows-Utopia,4134100,New York,40.732165,-73.787526,1.043136e+06,206095.5383,24.0,1341.0,4153657.0,4.071360e+09,Acer


In [ ]:
#create histogram of all trees based on NYC borough
fig = px.histogram(trees_df, 
                   x='borough',
                   color= 'Genus',
                   title='Histogram of NYC Trees',
                   labels={'borough': 'Borough'})
fig.update_layout(yaxis_title='Number of Trees')
fig.show()

## Create map of large trees in NYC

In [ ]:
import folium
from folium.plugins import MarkerCluster

In [ ]:
#create NYC map centered in NYC
nyc_map = folium.Map(location=[40.7128, -74.0060], zoom_start=11)

In [ ]:
#make a cluster
trees_cluster = MarkerCluster(name='NYC Tree').add_to(nyc_map)

In [ ]:
#select trees with diamater greater than 50 inches
largetrees_df = trees_df[trees_df.tree_dbh >= 50]

In [ ]:
#map large tree data frame, create popup including tree id, common species name, health and trunk diameter.
for idx, row in largetrees_df.iterrows():
    popup_text = f'<b>Tree# {row.tree_id}</b>\n Species:{row.spc_common}\n Health:{row.health}\n Truck Diameter,inches:{row.tree_dbh}'
    folium.Marker(
        location=[row.latitude, row.longitude],
        popup=popup_text,
        icon=folium.Icon(icon="glyphicon-tree-deciduous")
    ).add_to(trees_cluster)

In [ ]:
#show map
nyc_map